#  TP 2: 

### 18 mars 2024

### Par Samuel Fortin, Philippe Truchon et Benjamin Trudel

## TP2.1 Décomposition QR par la méthode Householder

#### Fonctions générales

In [281]:
import fnmatch
import functools
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math


sns.set_theme(style="ticks", palette="deep")

plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False



# Liste les noms de fichier d'un dossier
def listNameOfFiles(directory: str, extension="csv"):
    found_files = []
    for file in os.listdir(directory):
        if fnmatch.fnmatch(file, f"*.{extension}"):
            found_files.append(file)
    return found_files


# Lis et crée une matrice numpy à partir de chemin d'un fichier texte
def readTXT(path: str):
    fich = open(path, "r")
    fich_str = list(fich)
    fich.close()
    x = []
    for i in fich_str:
        elem_str = i.replace("\n", "")
        x.append(float(elem_str))
    return np.array(x)

sign = lambda x: math.copysign(1, x) # two will work

path = os.path.abspath("")
files_name = listNameOfFiles(path)


### Questions:

### a)

<i>À l’aide des équations (2.1.2) et (2.1.3), démontrez que les matrices de réflexion Q_i sont orthogonales.</i>

Sachant que $v^T v = I$:

\begin{gather*}
 H_{m,i}H_{m,i}^T = H_{m,i}H_{m,i}
 \\
 H_{m,i}H_{m,i}^T = \Big( I - \frac{2 v v^T}{v^T v} \Big) \Big( I - \frac{2 v v^T}{v^T v} \Big)
 \\
 H_{m,i}H_{m,i}^T = I - \frac{ 4 v v^T }{ v^T v } + \frac{ 4 v (v^T v) v^T }{ (v^T v)^2 }
 \\
 H_{m,i}H_{m,i}^T = I - \frac{ 4 v v^T }{ v^T v } + \frac{ 4 v v^T }{ v^T v }
 \\
 H_{m,i}H_{m,i}^T = I
\end{gather*}  


$$ Q_i = \left [
\begin{matrix}
I_i & 0 \\
0 & H_{m,i} 
\end{matrix}
\right ]$$

En utilisant la forme générale du produit des matrices par blocs, on sait que:

$$ M^T = \left [
\begin{matrix}
A^T & B^T \\
C^T & D^T 
\end{matrix}
\right ] $$

Donc:

$$ Q_i = \left [
\begin{matrix}
I_i & 0 \\
0 & H_{m,i} 
\end{matrix}
\right ] = \left [
\begin{matrix}
I_i^T & 0 \\
0 & H_{m,i}^T 
\end{matrix}
\right ] = Q_i^T$$





### b)

<i>Démontrez l’équation (2.1.5) et que la matrice Q est orthogonale.</i>

 

En utilisant la propriété d'associativité : $(AB)^T = B^T A^T$ et sachant que $Q_{i}^T Q_{i} = I$

\begin{gather*}
    Q = \prod_{i=0}^{n-1} Q_i^T
\end{gather*}

\begin{gather*}
    (\prod_{i=0}^{n-1} Q_i)^T (\prod_{i=0}^{n-1} Q_i) = (\prod_{i=n-1}^{0} Q_i^T)(\prod_{i=0}^{n-1} Q_i) = \prod_{i=0}^{n-1} I = I
\end{gather*}

La matrice Q est donc orthogonale.

### c)

<i>Implémentez la fonction householder_qr qui prend en argument une matrice A et qui retourne les matrices Q
et R obtenues par la méthode de Householder.</i>


In [282]:
def householder_qr(A, inter=False):
    m, n = A.shape
    Q = np.identity(m)
    for i in range(n):
        H = np.identity(m)
        x = A[i:, i]

        e1 = np.zeros(len(x)).T
        e1[0] = 1

        v = np.array([x]).T+np.copysign(np.linalg.norm(x), x[0]) * np.array([e1]).T


        Ht = np.identity(x.shape[0])

        Ht -= (2 * (v @ v.T)/(v.T @ v))

        H[i:, i:] = Ht
        Q = Q @ H
        A = H @ A
        if inter:
            print(f'A{i}:\n', A.round(6)) 
    return Q, A

### d)

<i>À l’aide d’une matrice de dimension 4 × 3 de votre choix, testez votre fonction householder_qr et comparez les
résultats obtenus avec ceux obtenus à l’aide de la fonction numpy.linalg.qr. Les matrices sont-elles exactement
les mêmes ? Si non, est-ce un problème?</i>



In [283]:
a = np.random.randint(1,10,(4,3))
print(a)
Q, R = householder_qr(a)
print('Q:\n', Q.round(6))
print('R:\n', R.round(6))
r2 = np.linalg.qr(a, mode='complete')
print(r2)

[[5 7 2]
 [5 5 4]
 [5 1 6]
 [6 4 3]]
Q:
 [[-0.474579 -0.674914 -0.043095 -0.563391]
 [-0.474579 -0.223616 -0.478351  0.704238]
 [-0.474579  0.67898  -0.409399 -0.382301]
 [-0.569495  0.182959  0.775704  0.201211]]
R:
 [[-10.535654  -8.447506  -7.403432]
 [  0.        -4.431663   2.378461]
 [  0.        -0.        -2.128875]
 [  0.        -0.        -0.      ]]
QRResult(Q=array([[-0.474579  , -0.67491386, -0.04309464, -0.56339055],
       [-0.474579  , -0.22361604, -0.47835053,  0.70423818],
       [-0.474579  ,  0.67897961, -0.4093991 , -0.38230073],
       [-0.5694948 ,  0.18295858,  0.77570355,  0.20121091]]), R=array([[-10.53565375,  -8.44750616,  -7.40343237],
       [  0.        ,  -4.4316633 ,   2.3784615 ],
       [  0.        ,   0.        ,  -2.12887531],
       [  0.        ,   0.        ,   0.        ]]))


Les matrices sont les mêmes, mais la matrice Q pourrait avoir des signes différents selon la convention de signe utilisée qui menerait à la même matrice R.

### e)

<i>À l’aide de la matrice utilisée en d, illustrez comment la multiplication successive des matrices Q_i triangularise
progressivement la matrice A. Dans l’élan, assurez-vous que les matrices Q et R obtenues sont bien orthogonale et
triangulaire supérieure, respectivement.</i>


In [284]:
Q, R = householder_qr(a, inter=True)
I = Q @ Q.T
print(I)

A0:
 [[-10.535654  -8.447506  -7.403432]
 [  0.         0.028369   0.973596]
 [  0.        -3.971631   2.973596]
 [  0.        -1.965957  -0.631685]]
A1:
 [[-10.535654  -8.447506  -7.403432]
 [  0.        -4.431663   2.378461]
 [  0.         0.         1.722573]
 [  0.        -0.        -1.250941]]
A2:
 [[-10.535654  -8.447506  -7.403432]
 [  0.        -4.431663   2.378461]
 [  0.        -0.        -2.128875]
 [  0.        -0.        -0.      ]]
[[ 1.00000000e+00  4.53915567e-17 -5.66396523e-17  8.45889556e-17]
 [ 4.53915567e-17  1.00000000e+00 -2.25141242e-16 -8.60167159e-17]
 [-5.66396523e-17 -2.25141242e-16  1.00000000e+00  1.19579461e-17]
 [ 8.45889556e-17 -8.60167159e-17  1.19579461e-17  1.00000000e+00]]


On oberve qu'à chaque itération la colonne la plus à gauche non triangulaire le devient en devenant des zéros en dessous de la diagonale. En même temps, on observe que la dernière matrice A qui est égale à R est triangulaire supérieur que tous les valeurs en dessous de la diagonale sont zéros. Finalement, on vérifie l'orthogonalité de Q en multipliant Q par sa transpose. Le résultat obtenu est la matrice identité comme prévu, confirmant l'orthogonalité. À noter, que les zéros dans ce cas ne sont pas exactement zéro à cause d'erreur numérique.

In [285]:
I[I< 1e-15] = 0
print(I)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## TP2.2 Mesures imprécises dans un jeu de bataille navale

### a)

<i>Modifiez votre code de décomposition QR pour qu’il retourne la décomposition QR réduite de la matrice d’entrée
lorsque l’argument additionnel reduite=True lui est passé.</i>

In [286]:
def householder_qr(A, inter=False, reduite=False):
    m, n = A.shape
    Q = np.identity(m)
    for i in range(n):
        H = np.identity(m)
        x = A[i:, i]

        e1 = np.zeros(len(x)).T
        e1[0] = 1

        v = np.array([x]).T+np.copysign(np.linalg.norm(x), x[0]) * np.array([e1]).T


        Ht = np.identity(x.shape[0])

        Ht -= (2 * (v @ v.T)/(v.T @ v))

        H[i:, i:] = Ht
        Q = Q @ H
        A = H @ A
        if inter:
            print(f'A{i}:\n', A.round(6)) 
    if reduite:
        A = A[:n,:]
    return Q, A

### b)

<i>Utilisez votre code pour résoudre approximativement l’équation (2.2.4). Vous utiliserez les données fournies dans
le fichier bataille_navale_equipeXX.csv où vous remplacerez XX par votre numéro d’équipe dans la boîte de
dépôt sur MonPortail. </i>

### c)

<i>Tracez les données (cercles noirs) et la solution estimée de la trajectoire(ligne pleine de la couleur de votre choix)
donnée par l’équation (2.2.3). </i>

### d)

<i>Obtenez la position d’impact du projectile (à y = 0) en résolvant l’équation quadratique (2.2.3) pour x à l’aide
d’une implémentation personnelle de la méthode de la bissection. Comparez votre solution avec celle obtenue en
résolvant cette même équation analytiquement. Considérant que votre embarcation se situe à la position (x, y) =
(0, 0), quelle est la distance horizontale vous séparant du point d’impact ?
 </i>